In [18]:

import requests
import pg8000
from shapely import wkb
import requests
import ssl

In [19]:
ssl_context = ssl.SSLContext()
ssl_context.verify_mode = ssl.CERT_NONE
conn = pg8000.connect(user = "p1711842", host = "bd-pedago.univ-lyon1.fr", port = 5432,
                              database = "p1711842", password = "Pu1WtY83xbUx", ssl_context=ssl_context)
localConnection = pg8000.connect(user = "postgres", host = "localhost", port = 5432,
                              database = "postgres", password = "password")

In [21]:
meteo_aeroports = conn.run("SELECT * from public.aeroport_proche_ville")
meteo_aeroports

(['0101000020E61000000C76C3B64519F73FF623456458CD4540',
  2972315,
  'Toulouse',
  'Lasbordes',
  'LFCL',
  4903.63627345],
 ['0101000020E61000005DBF60376CFB1E407BBDFBE3BD4A4840',
  2973783,
  'Strasbourg Neudorf',
  'Polygone',
  'LFGC',
  4015.27459378],
 ['0101000020E610000027C286A757CA0240280AF4893C6D4840',
  2988507,
  'Villacoublay',
  'Velizy',
  'LFPV',
  13908.56460239],
 ['0101000020E61000008D7A884677101D40126BF12900DA4540',
  2990440,
  'Nice',
  "Cote D\\\\'Azur",
  'LFMN',
  6405.29212013],
 ['0101000020E61000000AA2EE0390DAF8BF2B8716D9CE9B4740',
  2990969,
  'Nantes',
  'Nantes Atlantique',
  'LFRS',
  8338.73964086],
 ['0101000020E6100000FBCBEEC9C3020F40991249F432CE4540',
  2992166,
  'Montpellier',
  'Mediterranee',
  'LFMT',
  7976.8059459],
 ['0101000020E610000067F2CD36378615400B46257502A64540',
  2995469,
  'Marseille',
  'Provence',
  'LFML',
  20512.93108172],
 ['0101000020E6100000DEC83CF2076313409D2E8B89CDDF4640',
  2996944,
  'Lyon',
  'Bron',
  'LFLY',
  7933.188

In [22]:
aeroports = list(map(lambda x: [*x[1:], wkb.loads(x[0], hex=True)],
                     meteo_aeroports))

In [23]:
aeroports

[[2972315,
  'Toulouse',
  'Lasbordes',
  'LFCL',
  4903.63627345,
 [2973783,
  'Strasbourg Neudorf',
  'Polygone',
  'LFGC',
  4015.27459378,
 [2988507,
  'Villacoublay',
  'Velizy',
  'LFPV',
  13908.56460239,
 [2990440,
  'Nice',
  "Cote D\\\\'Azur",
  'LFMN',
  6405.29212013,
 [2990969,
  'Nantes',
  'Nantes Atlantique',
  'LFRS',
  8338.73964086,
 [2992166,
  'Montpellier',
  'Mediterranee',
  'LFMT',
  7976.8059459,
 [2995469,
  'Marseille',
  'Provence',
  'LFML',
  20512.93108172,
 [2996944,
  'Lyon',
  'Bron',
  'LFLY',
  7933.18880796,
 [2998324,
  'Lille',
  'Lesquin',
  'LFQQ',
  8192.89598752,
 [3031582,
  'Bordeaux',
  'Merignac',
  'LFBD',
  10734.39360187,

In [27]:
icao = list(set(map(lambda x : x[3], aeroports)))

In [28]:
url = 'https://api.met.no/weatherapi/tafmetar/1.0/taf.txt?icao={}'.format(",".join(icao))

headers = {
    'User-Agent': 'python',
    'Accept': 'text/plain'
}

response = requests.get(url, headers=headers)

In [29]:
tafs = response.text.split('\n')

In [30]:
tafs

['LFBD 301100Z 3012/3118 VRB03KT 9999 OVC007 TX10/3114Z TN05/3106Z BECMG 3018/3020 3000 BR TEMPO 3102/3112 -RADZ BKN005 BECMG 3103/3105 24010KT BECMG 3110/3112 32010KT 9999 NSW BKN015 TEMPO 3012/3018 BKN020=',
 'LFBD 301700Z 3018/3124 02005KT 9999 OVC008 TX10/3114Z TN03/3124Z BECMG 3018/3019 3000 BR TEMPO 3100/3109 1500 -RADZ BKN005 BECMG 3101/3103 24005KT BECMG 3109/3111 32010KT 9999 NSW SCT020 TEMPO 3112/3114 RADZ BKN010 BECMG 3114/3116 OVC030=',
 'LFBD 301910Z 3019/3124 02005KT 9999 OVC010 TX10/3114Z TN03/3124Z BECMG 3019/3021 3000 BR TEMPO 3100/3109 1500 -RADZ BKN005 BECMG 3101/3103 24005KT BECMG 3109/3111 32010KT 9999 NSW SCT020 TEMPO 3112/3114 RADZ BKN010 BECMG 3114/3116 OVC030=',
 'LFBD 302147Z 3021/3124 02005KT 9999 OVC015 TX10/3114Z TN03/3124Z TEMPO 3100/3109 1500 -RADZ BKN005 BECMG 3101/3103 24005KT BECMG 3109/3111 32010KT 9999 NSW SCT020 TEMPO 3112/3114 RADZ BKN010 BECMG 3114/3116 OVC030=',
 'LFBD 302300Z 3100/0106 02005KT 9999 OVC015 TX10/3114Z TN03/3124Z TEMPO 3100/3109 15

In [32]:
from functools import reduce

def reduceTafs(acc, curr):
    splitted = curr.split(' ')
    if splitted[0] != '':
        acc[splitted[0]] = {'ciel': None, 'taf': curr}
        if curr.find('SN') != -1:
            acc[splitted[0]]['ciel'] = 'neige'
        elif curr.find('RA') != -1:
            acc[splitted[0]]['ciel'] = 'pluie'
        elif curr.find('OVC') != -1:
            acc[splitted[0]]['ciel'] = 'nuageux'
        else:
            acc[splitted[0]]['ciel'] = 'soleil'
    return acc
tafReduced = reduce(reduceTafs,tafs,{})
tafReduced

{'LFBD': {'ciel': 'pluie',
  'taf': 'LFBD 310603Z 3106/0112 30005KT 9999 BKN014 OVC020 TX10/3114Z TN03/3124Z TEMPO 3106/3112 2000 -RADZ BKN003 BECMG 3112/3114 32012KT SCT020 BECMG 0102/0104 VRB02KT PROB30 0104/0108 MIFG='},
 'LFLY': {'ciel': 'pluie',
  'taf': 'LFLY 310500Z 3106/0106 16006KT 8000 OVC030 PROB30 TEMPO 3106/3109 4000 -RADZ BR BKN020 TEMPO 3110/3113 33015G25KT 4000 SHRA SCT020TCU OVC030 BECMG 3110/3112 33015KT TEMPO 3113/3118 34015G30KT 4000 SHRA SCT015TCU BKN030 BECMG 3122/3124 36005KT='},
 'LFQQ': {'ciel': 'pluie',
  'taf': 'LFQQ 310654Z 3106/0112 30018KT 9999 SCT030 TEMPO 3106/3119 30023G35KT FM312100 33012KT 9999 SCT040 BECMG 3122/0101 24010KT BECMG 0104/0107 BKN008 OVC013 TEMPO 0107/0112 3000 DZRA OVC005='},
 'LFRS': {'ciel': 'soleil',
  'taf': 'LFRS 310500Z 3106/0112 29008KT 9999 SCT010 BKN025 TEMPO 3106/3108 BKN010 TEMPO 3109/3117 33015G25KT='}}

In [37]:
from pprint import pprint
from yattag import Doc, indent

doc, tag, text = Doc().tagtext()


doc.asis('<?xml version="1.0" encoding="UTF-8"?>')
with tag('kml', xmlns="http://www.opengis.net/kml/2.2"):
    with tag('Document'):
        for aero in aeroports:
            with tag('Placemark'):
                with tag('name'):
                    text(aero[1])
                with tag('description'):
                    text(tafReduced[aero[3]]['taf'] if aero[3] in tafReduced else '')
                with tag('Point'):
                    with tag('coordinates'):
                        text("{},{}".format(aero[5].xy[0][0], aero[5].xy[1][0]))
                with tag('Style'):
                    with tag('IconStyle'):
                        with tag('scale'):
                            text('3')
                        with tag('Icon'):
                            with tag('href'):
                                if(aero[3] in tafReduced):
                                    temps = tafReduced[aero[3]]['ciel']
                                    if(temps == 'soleil'):
                                        text('https://s1.qwant.com/thumbr/0x380/2/8/5c823871ecb28b140cce1d49f4457c786696a332c08420b1c58c69cda8a875/sunicon.png?u=https%3A%2F%2Fopenclipart.org%2Fimage%2F2400px%2Fsvg_to_png%2F208526%2Fsunicon.png&q=0&b=1&p=0&a=0')
                                    elif(temps == 'nuageux'):
                                        text('https://s1.qwant.com/thumbr/0x380/1/9/fa85ef10ef0acda773a86c40f0794b8fd59f2b7ec95bb42c18cbc06a370750/cloud-icon.png?u=http%3A%2F%2Ficons.iconarchive.com%2Ficons%2Fpaomedia%2Fsmall-n-flat%2F1024%2Fcloud-icon.png&q=0&b=1&p=0&a=0')
                                    elif(temps == 'neige'):
                                        text('http://franck.favetta.free.fr/cours/lyon1/tiwsig/exercices/meteo/images/icones/neige.png')
                                    elif(temps == 'pluie'):
                                        text('http://franck.favetta.free.fr/cours/lyon1/tiwsig/exercices/meteo/images/icones/pluie.png')





textfile = open("output.kml", "w")
a = textfile.write((indent(doc.getvalue())))
textfile.close()
pprint(indent(doc.getvalue()))

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<kml xmlns="http://www.opengis.net/kml/2.2">\n'
 '  <Document>\n'
 '    <Placemark>\n'
 '      <name>Toulouse</name>\n'
 '      <description></description>\n'
 '      <Point>\n'
 '        <coordinates>1.44367,43.60426</coordinates>\n'
 '      </Point>\n'
 '      '
 '<Style><IconStyle><scale>3</scale><Icon><href></href></Icon></IconStyle></Style>\n'
 '    </Placemark>\n'
 '    <Placemark>\n'
 '      <name>Strasbourg Neudorf</name>\n'
 '      <description></description>\n'
 '      <Point>\n'
 '        <coordinates>7.74553,48.58392</coordinates>\n'
 '      </Point>\n'
 '      '
 '<Style><IconStyle><scale>3</scale><Icon><href></href></Icon></IconStyle></Style>\n'
 '    </Placemark>\n'
 '    <Placemark>\n'
 '      <name>Villacoublay</name>\n'
 '      <description></description>\n'
 '      <Point>\n'
 '        <coordinates>2.3488,48.85341</coordinates>\n'
 '      </Point>\n'
 '      '
 '<Style><IconStyle><scale>3</scale><Icon><href></href></Icon>